# OD Matrix

This notebook creates a graph for each territory and generate the OD matrix of the shortest path between each OD pair.

The input information for this script is the graph's links and nodes of each territory previusly generated by the script "Graph_Data". The node information contains the centroids of the census blocks as the origins, the opportunities as the destinations, and the nodes of the street network . The "edge" information contains the links that connect the origins with the street network, the links that connect the destinations with the street network, and the links of the street network.

In [ ]:
import pandas as pd
import geopandas as gpd

import utils.analysis as an

In [ ]:
# Import the names of the territories
l_names = pd.read_csv('../data/input/table/mpios_names.txt',header=None)
l_names = list(l_names[0])

# Types of codes
codes_names_list = ['health','sport','education','financial','cultural','parks']

In [ ]:
for m in l_names:
    # Import nodes and edges of the graph
    nodes = gpd.read_file(f'../data/output/shape/graphs/nodes/{m}_graph_nodes.shp')
    edges = gpd.read_file(f'../data/output/shape/graphs/links/{m}_graph_links.shp')

    # Para eliminar
    nodes['Type'] = nodes['Type'].replace('street', 'network')
    edges['Type'] = edges['Type'].replace('street', 'network')
    edges = edges.rename(columns={'time':'weight'})

    for c in codes_names_list:
        # Create subfiles to filter the destinations according to the type
        
        # Change the current type for 'destin'
        nodes['Type'] = nodes['Type'].replace(c, 'destin')

        # OD matrix calculation
        OD = an.OD_matrix_igraph(nodes, edges)

        # # Export results
        OD.to_csv(f'../data/output/table/OD_dfs_/{l_names[m]}_OD_df_{c}.txt',index=False, header=True)

        # Change 'destin' for the current type
        nodes['Type'] = nodes['Type'].replace('destin', c)